In [6]:
from langchain_community.chat_models.tongyi import ChatTongyi
from langchain.messages import HumanMessage

model = ChatTongyi(
    streaming=True,
    api_key = "sk-43070f4cd1074965a93a03d6d5333cd8",
    model="qwen-flash",
)


## 调用大模型

### 普通调用

In [8]:
# 使用 invoke() 并传入一条消息或一条消息列表。
from langchain.messages import HumanMessage, AIMessage, SystemMessage

response = model.invoke("Why do parrots have colorful feathers?")
print(response)
#
# conversation = [
#     {"role": "system", "content": "You are a helpful assistant that translates English to French."},
#     {"role": "user", "content": "Translate: I love programming."},
#     {"role": "assistant", "content": "J'adore la programmation."},
#     {"role": "user", "content": "Translate: I love building applications."}
# ]
#
# response = model.invoke(conversation)
# print(response)  # AIMessage("J'adore créer des applications.")

content='Parrots have colorful feathers for several important evolutionary reasons, primarily related to survival, communication, and reproduction. Here are the main reasons:\n\n1. **Mate Attraction (Sexual Selection):**  \n   Bright and vibrant colors play a key role in attracting mates. In many parrot species, males with more vivid plumage are seen as healthier, stronger, or genetically superior. This makes them more likely to be chosen by females during mating, increasing their chances of passing on their genes.\n\n2. **Species Recognition:**  \n   The unique color patterns help parrots identify members of their own species, especially in dense forests where visibility is limited. This ensures they can find suitable mates and avoid confusion with other species.\n\n3. **Social Communication:**  \n   Parrots use their colors to communicate within flocks. Changes in feather coloration—such as fluffing up or brightening—can signal mood, dominance, aggression, or submission. These visual

In [9]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

conversation = [
    SystemMessage("You are a helpful assistant that translates English to French."),
    HumanMessage("Translate: I love programming."),
    AIMessage("J'adore la programmation."),
    HumanMessage("Translate: I love building applications.")
]

response = model.invoke(conversation)
print(response)  # AIMessage("J'adore créer des applications.")

content="J'adore créer des applications." additional_kwargs={} response_metadata={'finish_reason': 'stop', 'request_id': 'f174a565-8bf9-4f44-a1cf-56a77332f9ca', 'token_usage': {'input_tokens': 55, 'output_tokens': 8, 'total_tokens': 63, 'prompt_tokens_details': {'cached_tokens': 0}}} id='lc_run--9de5775c-90f6-4908-9a28-0f0d1fb5f42f'


### 流式调用

In [10]:
for chunk in model.stream("Why do parrots have colorful feathers?"):
    print(chunk.text, end="|", flush=True)

Par|rots| have| colorful| feathers for several important| evolutionary reasons, primarily| related to survival,| communication, and reproduction|. Here are the| main reasons:

1|. **Mate At|traction (Sexual| Selection):**  
|   Bright and vivid| colors in parrots| are often used to| attract mates. In| many species, more| vibrant plumage signals| good health, strong| genes, and overall| fitness. For example|, male parrots| with brighter feathers may| be more likely to| win the attention of| females during courtship| displays.

2.| **Species and Individual| Recognition:**  
  | The unique color patterns| help parrots identify| members of their own| species and distinguish individuals| within a flock.| This is crucial for| social bonding, avoiding| hybridization, and| maintaining group cohesion.

|3. **Social| Communication:**  
  | Parrots use their| feather colors to communicate| emotions and intentions.| For instance, changes| in color intensity or| specific patterns can signal| aggres

In [11]:
full = None  # None | AIMessageChunk
for chunk in model.stream("What color is the sky?"):
    full = chunk if full is None else full + chunk
    print(full.text)

# The
# The sky
# The sky is
# The sky is typically
# The sky is typically blue
# ...

print(full.content_blocks)
# [{"type": "text", "text": "The sky is typically blue..."}]

The
The sky
The sky is
The sky is typically
The sky is typically blue during the day
The sky is typically blue during the day due to Rayleigh
The sky is typically blue during the day due to Rayleigh scattering, which causes
The sky is typically blue during the day due to Rayleigh scattering, which causes shorter blue wavelengths of
The sky is typically blue during the day due to Rayleigh scattering, which causes shorter blue wavelengths of sunlight to scatter more
The sky is typically blue during the day due to Rayleigh scattering, which causes shorter blue wavelengths of sunlight to scatter more than other colors.
The sky is typically blue during the day due to Rayleigh scattering, which causes shorter blue wavelengths of sunlight to scatter more than other colors. However, the color
The sky is typically blue during the day due to Rayleigh scattering, which causes shorter blue wavelengths of sunlight to scatter more than other colors. However, the color of the sky can
The sky is typic

In [12]:
# 流媒体事件
async for event in model.astream_events("Hello"):

    if event["event"] == "on_chat_model_start":
        print(f"Input: {event['data']['input']}")

    elif event["event"] == "on_chat_model_stream":
        print(f"Token: {event['data']['chunk'].text}")

    elif event["event"] == "on_chat_model_end":
        print(f"Full message: {event['data']['output'].text}")

    else:
        pass

Input: Hello
Token: Hello
Token: !
Token:  How
Token:  can
Token:  I assist you today
Token: ? 😊
Token: 
Token: 
Full message: Hello! How can I assist you today? 😊


### 批量请求大模型

In [13]:
responses = model.batch([
    "Why do parrots have colorful feathers?",
    "How do airplanes fly?",
    "What is quantum computing?"
])
for response in responses:
    print(response)

content="Parrots have colorful feathers for several important evolutionary reasons, primarily related to survival, communication, and reproduction. Here are the main reasons:\n\n1. **Mate Attraction (Sexual Selection):**  \n   Bright, vivid colors in parrots often play a key role in attracting mates. In many species, more vibrant plumage signals good health, strong genes, and high fitness. For example, males with brighter feathers may be more likely to attract females, increasing their chances of successful mating.\n\n2. **Species Recognition:**  \n   The unique color patterns help parrots identify members of their own species, especially in diverse environments where multiple bird species live together. This prevents confusion during social interactions and breeding.\n\n3. **Social Communication:**  \n   Parrots use coloration to communicate with each other. Changes in feather color (such as puffing up or shifting hues) can signal emotions like aggression, fear, excitement, or submiss

In [14]:
for response in model.batch_as_completed([
    "Why do parrots have colorful feathers?",
    "How do airplanes fly?",
    "What is quantum computing?"
]):
    print(response)

(0, AIMessage(content="Parrots have colorful feathers for several important evolutionary reasons, primarily related to survival, communication, and reproduction. Here are the main reasons:\n\n1. **Mate Attraction (Sexual Selection):**  \n   Bright and vibrant colors in parrots often play a key role in attracting mates. In many species, males with more vivid plumage are seen as healthier, stronger, or genetically superior, making them more desirable partners. This is a classic example of sexual selection—where traits that enhance mating success are favored over time.\n\n2. **Species Recognition:**  \n   The unique color patterns help parrots identify members of their own species, especially in areas where multiple parrot species live together. This prevents confusion during mating and social interactions.\n\n3. **Social Communication:**  \n   Coloration can signal mood, intentions, or social status within a flock. For example, a parrot might change the appearance of its feathers (throug

In [ ]:
list_of_inputs = [
    "Why do parrots have colorful feathers?",
    "How do airplanes fly?",
    "What is quantum computing?"
]

model.batch(
    list_of_inputs,
    config={
        'max_concurrency': 5,  # Limit to 5 parallel calls
    }
)

## 工具调用

### 一般工具调用

In [15]:
from langchain.tools import tool

@tool
def get_weather(location: str) -> str:
    """Get the weather at a location."""
    return f"It's sunny in {location}."


model_with_tools = model.bind_tools([get_weather])

response = model_with_tools.invoke("What's the weather like in Boston?")
for tool_call in response.tool_calls:
    # View tool calls made by the model
    print(f"Tool: {tool_call['name']}")
    print(f"Args: {tool_call['args']}")

Tool: get_weather
Args: {'location': 'Boston'}


### 工具执行循环

In [16]:
# Bind (potentially multiple) tools to the model
model_with_tools = model.bind_tools([get_weather])

# Step 1: Model generates tool calls
messages = [{"role": "user", "content": "What's the weather in Boston?"}]
ai_msg = model_with_tools.invoke(messages)
messages.append(ai_msg)

# Step 2: Execute tools and collect results
for tool_call in ai_msg.tool_calls:
    # Execute the tool with the generated arguments
    tool_result = get_weather.invoke(tool_call)
    messages.append(tool_result)

# Step 3: Pass results back to model for final response
final_response = model_with_tools.invoke(messages)
print(final_response.text)
# "The current weather in Boston is 72°F and sunny."

The weather in Boston is sunny.


### 强制工具调用

In [ ]:
model_with_tools = model.bind_tools([tool_1], tool_choice="tool_1")
model_with_tools = model.bind_tools([tool_1], tool_choice="any")

## 结构化输出

### JSON

In [8]:
import json

json_schema = {
    "title": "Movie",
    "description": "A movie with details",
    "type": "object",
    "properties": {
        "title": {
            "type": "string",
            "description": "The title of the movie"
        },
        "year": {
            "type": "integer",
            "description": "The year the movie was released"
        },
        "director": {
            "type": "string",
            "description": "The director of the movie"
        },
        "rating": {
            "type": "number",
            "description": "The movie's rating out of 10"
        }
    },
    "required": ["title", "year", "director", "rating"]
}

model_with_structure = model.with_structured_output(
    json_schema,
)
response = model_with_structure.invoke("Provide details about the movie Inception")
print(response)  # {'title': 'Inception', 'year': 2010, ...}

{'title': 'Inception', 'year': 2010, 'director': 'Christopher Nolan', 'rating': 8.8}


### pydantic 

In [9]:
from pydantic import BaseModel, Field

class Movie(BaseModel):
    """A movie with details."""
    title: str = Field(..., description="The title of the movie")
    year: int = Field(..., description="The year the movie was released")
    director: str = Field(..., description="The director of the movie")
    rating: float = Field(..., description="The movie's rating out of 10")

model_with_structure = model.with_structured_output(Movie)
response = model_with_structure.invoke("Provide details about the movie Inception")
print(response)  # Movie(title="Inception", year=2010, director="Christopher Nolan", rating=8.8)

title='Inception' year=2010 director='Christopher Nolan' rating=8.8


### TypedDict

In [10]:
from typing_extensions import TypedDict, Annotated

class MovieDict(TypedDict):
    """A movie with details."""
    title: Annotated[str, ..., "The title of the movie"]
    year: Annotated[int, ..., "The year the movie was released"]
    director: Annotated[str, ..., "The director of the movie"]
    rating: Annotated[float, ..., "The movie's rating out of 10"]

model_with_structure = model.with_structured_output(MovieDict)
response = model_with_structure.invoke("Provide details about the movie Inception")
print(response)  # {'title': 'Inception', 'year': 2010, 'director': 'Christopher Nolan', 'rating': 8.8}

{'title': 'Inception', 'year': 2010, 'director': 'Christopher Nolan', 'rating': 8.8}


In [7]:

from langchain.chat_models import init_chat_model
from langchain_core.callbacks import UsageMetadataCallbackHandler

from langchain_community.chat_models.tongyi import ChatTongyi
from langchain.messages import HumanMessage

model = ChatTongyi(
    streaming=True,
    api_key = "sk-43070f4cd1074965a93a03d6d5333cd8",
    model="qwen-flash",
)
result_1 = model.invoke("Hello")
print(result_1.response_metadata['token_usage'])

{'input_tokens': 9, 'output_tokens': 11, 'total_tokens': 20, 'prompt_tokens_details': {'cached_tokens': 0}}
